In [1]:
from haystack.document_stores import OpenSearchDocumentStore

doc_store = OpenSearchDocumentStore()

/Users/tuanacelik/opt/anaconda3/envs/sagemeker/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/tuanacelik/opt/anaconda3/envs/sagemeker/lib/python3.11/site-packages/opensearchpy/connection/http_urllib3.py:199: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(
/Users/tuanacelik/opt/anaconda3/envs/sagemeker/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/tuanacelik/opt/anaconda3/envs/sagemeker/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified 

In [3]:
doc_store.get_all_documents()[0]

/Users/tuanacelik/opt/anaconda3/envs/sagemeker/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/tuanacelik/opt/anaconda3/envs/sagemeker/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/tuanacelik/opt/anaconda3/envs/sagemeker/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-war

<Document: {'content': 'Most OpenSearch configuration can take place in the cluster settings API. Certain operations require you to modify opensearch.yml and restart the cluster.\nWhenever possible, use the cluster settings API instead; opensearch.yml is local to each node, whereas the API applies the setting to all nodes in the cluster. Certain settings, however, require opensearch.yml. In general, these settings relate to networking, cluster formation, and the local file system. To learn more, see Cluster formation.\n', 'content_type': 'text', 'score': None, 'meta': {'_split_id': 0, '_split_overlap': [{'range': [366, 498], 'doc_id': '33086535ace8e57f75c09936500cba93'}], 'title': 'Configuring OpenSearch', 'ancestors': ['Install and upgrade'], 'type': 'DOCS', 'url': '/docs/latest/install-and-configure/configuration/'}, 'id_hash_keys': ['content'], 'embedding': None, 'id': 'd8a3b4091660339fe9b280f9a8a30884'}>

In [5]:
from haystack.nodes import EmbeddingRetriever, PromptTemplate, PromptNode

retriever = EmbeddingRetriever(document_store=doc_store, embedding_model="sentence-transformers/multi-qa-mpnet-base-dot-v1", devices=["mps"])
doc_store.update_embeddings(retriever)

/Users/tuanacelik/opt/anaconda3/envs/sagemeker/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
Updating embeddings:   0%|          | 0/8841 [00:00<?, ? Docs/s]/Users/tuanacelik/opt/anaconda3/envs/sagemeker/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/tuanacelik/opt/anaconda3/envs/sagemeker/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: http

KeyboardInterrupt: 

In [ ]:
from getpass import getpass

api_key = getpass("Enter model provider API key:")

In [ ]:
gen_qa_with_references = PromptNode(default_prompt_template="question-answering-with-references", model_name_or_path="gpt-4", api_key=api_key)